# Import thư viện

In [1]:
import numpy as np
import pandas as pd
import ast
import re
from tqdm.auto import tqdm

c:\Users\ADMIN\Desktop\Data\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Làm sạch từng nguồn và hợp nhất

Các thao tác làm sạch bao gồm:
- Loại bỏ các dòng null và duplicate
- Loại bỏ các thông tin gây nhiễu như: các kí tự lặp lại (........, ----, "\n\n\n\n",...)
- Loại bỏ các thông tin cá nhân, tổ chức như: email, gmail, yahoo, số điện thoại,...
- Loại bỏ các conversation (hội thoại) mà rỗng, không có câu hỏi hoặc câu trả lời

In [2]:
def drop_noise(text):
    # Loại bỏ các chuỗi lặp lại từ 3 lần trở lên và thay thế bằng 1 lần
    text = re.sub(r'(.)\1{2,}', r'\1', text)
    
    # Thay thế các ký tự xuống dòng liên tục (\n\n\n\n) bằng 1 \n
    text = re.sub(r'\n\s*\n+', '\n', text)
    return text
    
def drop_personal_information(text):
    # Loại bỏ email (gmail, yahoo,...)
    text = re.sub(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}\b', '', text)
    
    # Loại bỏ số điện thoại (các chuỗi gồm 10-11 số liên tiếp)
    text = re.sub(r'\b\d{10,11}\b', '', text)
    
    return text

def str2lst_of_dict(text):
    try:
        string = text.replace("\n", "\\n")

        # Bây giờ dùng ast.literal_eval
        data = ast.literal_eval(string)
        return data
    except:
        print("Phát hiện lỗi!:", text)
        return None
    
def conversation_processing(conversation):
    for message in conversation:
        if message["message"] is None or message["message"] == "null":
            return None
        message["message"] = drop_noise(drop_personal_information(message["message"]))
    return conversation    

In [3]:
csv_path = ["bv115_data_crawling.csv","bvnd2_data_crawling.csv","eDoctor_data_crawling.csv", "Express_data_crawling.csv", "Ivie_data_crawling.csv", "VietSkin_data_crawling.csv"]
data_dict = {
    "source": [],
    "url": [],
    "conversation": []
}
for path in csv_path:
    df = pd.read_csv(path)
    for i in tqdm(range(df.shape[0]), desc=f"Processing {path}:"):
        data = df.iloc[i]
        source = data.source
        url = data.url
        conversation_text = data.conversation
        conversation = str2lst_of_dict(conversation_text)
        if conversation is None:
            continue
        conversation = conversation_processing(conversation)
        if conversation is None:
            continue
        data_dict["source"].append(source)
        data_dict["url"].append(url)
        data_dict["conversation"].append(conversation)
all_df = pd.DataFrame(data_dict)

Processing eDoctor_data_crawling.csv:: 100%|██████████| 371/371 [00:00<00:00, 5610.55it/s]


Phát hiện lỗi!: [{"role": "Người dùng", "message": null}, {"role": "Bác sĩ", "message": null}]
Phát hiện lỗi!: [{"role": "Người dùng", "message": null}, {"role": "Bác sĩ", "message": null}]
Phát hiện lỗi!: [{"role": "Người dùng", "message": null}, {"role": "Bác sĩ", "message": null}]
Phát hiện lỗi!: [{"role": "Người dùng", "message": null}, {"role": "Bác sĩ", "message": null}]
Phát hiện lỗi!: [{"role": "Người dùng", "message": null}, {"role": "Bác sĩ", "message": null}]
Phát hiện lỗi!: [{"role": "Người dùng", "message": null}, {"role": "Bác sĩ", "message": null}]
Phát hiện lỗi!: [{"role": "Người dùng", "message": null}, {"role": "Bác sĩ", "message": null}]
Phát hiện lỗi!: [{"role": "Người dùng", "message": null}, {"role": "Bác sĩ", "message": null}]
Phát hiện lỗi!: [{"role": "Người dùng", "message": null}, {"role": "Bác sĩ", "message": null}]
Phát hiện lỗi!: [{"role": "Người dùng", "message": null}, {"role": "Bác sĩ", "message": null}]
Phát hiện lỗi!: [{"role": "Người dùng", "message":

Processing VietSkin_data_crawling.csv:: 100%|██████████| 6/6 [00:00<00:00, 6001.87it/s]


In [4]:
all_df.shape

(1057, 3)

In [5]:
all_df.head()

,source,url,conversation
0,Bệnh viện 115,https://benhvien115.com.vn/tu-van-bac-si/ths-b...,"[{'role': 'Người dùng', 'message': 'Dạ em chào..."
1,Bệnh viện 115,https://benhvien115.com.vn/tu-van-bac-si/ths-b...,"[{'role': 'Người dùng', 'message': 'Chào BS, M..."
2,Bệnh viện 115,https://benhvien115.com.vn/tu-van-bac-si/ths-b...,"[{'role': 'Người dùng', 'message': 'BS ơi giúp..."
3,Bệnh viện 115,https://benhvien115.com.vn/tu-van-bac-si/ths-b...,"[{'role': 'Người dùng', 'message': 'Mông của e..."
4,Bệnh viện 115,https://benhvien115.com.vn/tu-van-bac-si/ths-b...,"[{'role': 'Người dùng', 'message': 'Chào BS, E..."


In [6]:
all_df.to_csv("data_crawling.csv",index=False)